In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.interpolate import interp1d
import statsmodels.api as sm
import pandas as pd
import numpy as np
import itertools

C:\Users\conor\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:49: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)


In [2]:
pick_values = pd.read_csv('https://raw.githubusercontent.com/mcqconor/draftvalue/main/pick_val.csv')

In [38]:
picks = pick_values['pick_num'].values
jj_vals = pick_values['jj_val'].values
fs_vals = pick_values['otc_val'].values

In [4]:
# lowess will return our "smoothed" data with a y value for at every x-value
lowess_jj = sm.nonparametric.lowess(jj_vals, picks, frac = 0.01)
lowess_fs = sm.nonparametric.lowess(fs_vals, picks, frac = 0.01)

# unpack the lowess smoothed points to their values
lowess_jj_x = list(zip(*lowess_jj))[0]
lowess_jj_y = list(zip(*lowess_jj))[1]

lowess_fs_x = list(zip(*lowess_fs))[0]
lowess_fs_y = list(zip(*lowess_fs))[1]

f_jj = interp1d(lowess_jj_x, lowess_jj_y, bounds_error=False)
f_fs = interp1d(lowess_fs_x, lowess_fs_y, bounds_error=False)

In [39]:
f_jj_inv = interp1d(jj_vals, picks, bounds_error=False)
f_fs_inv = interp1d(fs_vals, picks, bounds_error = False)

In [6]:
f_fs(8)

array(1946.)

In [7]:
team_picks = [11,42,73,111,187,228]
opp_picks = [8,43, 58, 74, 112, 150, 188, 212]

target = -250
pick_val = f_jj(8)

In [8]:
numbers = [x*-1 for x in f_jj(team_picks).tolist()] + f_jj(opp_picks).tolist()

result = [seq for i in range(len(numbers), 0, -1)
          for seq in itertools.combinations(numbers, i)
          if (pick_val in seq) & (sum(seq) <= target)]

In [9]:
trade_options = []
for i in result:
    i = list(i)
    for j in range(len(i)):
        if i[j] <0:
            i[j] = -1*i[j]
    
    trade_options = trade_options + [i]

In [32]:
trade_jj_vals = []
for res in result:
    trade_jj_vals = trade_jj_vals + [sum(res)]

In [10]:
trade_picks = []
for trade in trade_options:
    trade_picks = trade_picks + [list(f_jj_inv(trade))]

In [11]:
trade_ids = []
for trade in trade_picks:
    trade_ids.append(','.join(map(str, (int(i) for i in trade))))

In [43]:
fs_vals = []
for trade in trade_picks:
    t_vals = []
    for pick in trade:
        if team_picks.count(pick):
            val = -1*f_fs(pick)
        
        elif pick == 252 and not team_picks.count(pick):
            val = -1*f_fs(228)
            
        else:
            val = f_fs(pick)
            
        t_vals = t_vals + [val]
        
    fs_vals.append(sum(t_vals))

In [46]:
fs_pick_equiv = f_fs_inv([abs(x) for x in fs_vals])
jj_pick_equiv = f_jj_inv([abs(x) for x in trade_jj_vals])

In [48]:
trade_df = pd.DataFrame({
    'trade_id':trade_ids,
    'jj_vals':trade_jj_vals,
    'jj_pick':jj_pick_equiv,
    'fs_vals':fs_vals,
    'fs_pick_equiv':fs_pick_equiv
})

trade_df.to_csv('was_atl_8.csv')